In [3]:
!/gws/pw/j05/cop26_hackathons/bristol/install-kernel

Installed kernelspec cmip6 in /home/users/fdmv/.local/share/jupyter/kernels/cmip6
Installed cmip6 Jupyter kernel for the hackathon


In [7]:
def compute_linear_interp_master(expc_data_loc,expc_data_name,depth_exp,depth_transf):

	# INPUTS
	# expc_data_loc - full path to netcdf file
	# expc_data_name - netcdf file name
	# depth_exp: depth of export (e.g. 100m)
	# depth_transf: depth of export (e.g. 1000m)

	# OUTPUTS
	# flux_exp_interp.nc: export flux at "depth_exp" metres
	# flux_transf_interp.nc: transfer flux at "depth_flux" metres


    print('>>> loading data')
    ### Loading data
    dataset=xr.open_mfdataset(expc_data_loc+expc_data_name)

    ### getting arrays
    # Computing linearly interpolated fluxes

#    depth_exp=100 # export depth of interest (in metres)
#    depth_transf=1000 # transfer depth of interest (in metres)

    print('>>> interpolating data (sloooow)')
    ### annual averaging
    lev_exp_prev = dataset.sel(lev=depth_exp,method="nearest").compute()['lev'] # nearest export level above z = depth
    lev_exp_next = dataset.sel(lev=depth_exp,method="backfill").compute()['lev'] # nearest export level below z = depth

    lev_transf_prev = dataset.sel(lev=depth_transf,method="nearest").compute()['lev'] # nearest transfer level above z = depth
    lev_transf_next = dataset.sel(lev=depth_transf,method="backfill").compute()['lev'] # nearest transfer level below z = depth

    flux_exp_prev = dataset.sel(lev=depth_exp, method="nearest").compute()['expc'] # export flux at nearest level above z = depth
    flux_exp_next = dataset.sel(lev=depth_exp, method="backfill").compute()['expc'] # export flux at nearest level below z = depth

    flux_transf_prev = dataset.sel(lev=depth_transf, method="nearest").compute()['expc'] # tranfer flux at nearest level above z = depth
    flux_transf_next = dataset.sel(lev=depth_transf, method="backfill").compute()['expc'] # transfer flux at nearest level below z = depth

    # computing the linearest interpolated flux at z= depth
    interp_par_exp = (depth_exp - lev_exp_prev)/(lev_exp_next - lev_exp_prev)
    interp_par_transf = (depth_transf - lev_transf_prev)/(lev_transf_next - lev_transf_prev)
    flux_exp_interp = (flux_exp_prev*(1-interp_par_exp) + flux_exp_next*interp_par_exp)
    flux_transf_interp = (flux_transf_prev*(1-interp_par_transf) + flux_transf_next*interp_par_transf)
    
    print('>>> output data')
    flux_exp_interp.to_netcdf('flux_'+str(depth_exp)+'m'+'_interp_'+expc_data_name)
    flux_transf_interp.to_netcdf('flux_'+str(depth_transf)+'m'+'_interp_'+expc_data_name)
    
###### END ######

In [6]:
# Loading modules, etc
from itertools import chain
from glob import glob

import matplotlib.pyplot as plt
import xarray as xr

compute_linear_interp_master('/gws/pw/j05/cop26_hackathons/bristol/project03/input_nc/UKESM1-0-LL/','expc_Omon_UKESM1-0-LL_piControl_r1i1p1f2_gn_196001-199912.nc',100,1000)

>>> loading data
>>> interpolating data (sloooow)
>>> output data
